In [ ]:
## Add in SalePrice and YrSold cols.
features = no_reno_features + ['YrSold','SalePrice']

## TRAININGDATA from Residential_log
TRAININGDATA = residential_log[features].copy()

## Dummy out
TRAININGDATA = pd.get_dummies(TRAININGDATA, columns=no_reno_features_cat, drop_first=True)

print TRAININGDATA.shape
##SPLIT THE DATA INTO < 2010 AND == 2010
TESTSET = TRAININGDATA[TRAININGDATA.YrSold == 2010].copy()
TRAINSET = TRAININGDATA[TRAININGDATA.YrSold < 2010].copy()

## YrSold served is Purpose, we can drop it.
TESTSET.drop('YrSold', axis=1, inplace=True)
TRAINSET.drop('YrSold', axis=1, inplace=True)



In [ ]:
ss = StandardScaler()

In [ ]:
## TRAINING SET
X_train = TRAINSET.drop('SalePrice', axis=1)
X_train = ss.fit_transform(X_train) ##Standardize
y_train = TRAINSET['SalePrice'].values

## TESTING SET
X_test = TESTSET.drop('SalePrice', axis=1)
X_test = ss.fit_transform(X_test) ##Standardize
y_test = TESTSET['SalePrice'].values

### Base LinearRegression cv=15

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
linreg_scores = cross_val_score(model, X_test, y_test, cv=15)
## Tried multiple Cvs, 15 isnt the best, at 14: -5.86784329765e+25, then at 15: 0.794398096911 

#print linreg_scores
print('Base Test LinearRegression R2 score: {}'.format(np.mean(linreg_scores)))
## Can't trust this Base Test

### RidgeCV cv=15

In [ ]:
ridge_alphas = np.logspace(-2, 7, 50)

optimal_ridge = RidgeCV(alphas=ridge_alphas, cv=15)
optimal_ridge.fit(X_train, y_train)

print('Optimal alpha: {}'.format(optimal_ridge.alpha_))

### Ridge with optimal_alpha cv=15

In [ ]:
ridge = Ridge(alpha=optimal_ridge.alpha_)

ridge_scores = cross_val_score(ridge, X_train, y_train, cv=15)

#print ridge_scores
print('Ridge R2 score: {}'.format(np.mean(ridge_scores)))

### LassoCV cv=15

In [ ]:
optimal_lasso = LassoCV(n_alphas=500, cv=15, verbose=1)
optimal_lasso.fit(X_train, y_train)

print optimal_lasso.alpha_

###  Lasso with optimal_alpha cv=15

In [ ]:
lasso = Lasso(alpha=optimal_lasso.alpha_)

lasso_scores = cross_val_score(lasso, X_train, y_train, cv=15)

#print lasso_scores
print('Lasso R2 score: {}'.format(np.mean(lasso_scores)))

### Lets try ElasticNet cv=15

In [ ]:
l1_ratios = np.linspace(0.01, 1.0, 25)
optimal_enet = ElasticNetCV(l1_ratio=l1_ratios, n_alphas=30, cv=15,
                            verbose=1)
optimal_enet.fit(X_train, y_train)

print optimal_enet.alpha_
print optimal_enet.l1_ratio_

In [ ]:
## The L1 Ratio is 1.0 = LASSO
## Lets see with the optimals
enet = ElasticNet(alpha=optimal_enet.alpha_, l1_ratio=optimal_enet.l1_ratio_)

enet_scores = cross_val_score(enet, X_train, y_train, cv=15)

print enet_scores
print np.mean(enet_scores)

### Seems to be telling me .....

In [ ]:
## Plot to see the Residuals
ridge.fit(X_train, y_train)
lasso.fit(X_train, y_train)

In [ ]:
# Model residuals:
ridge_resid = y_test - ridge.predict(X_test)
lasso_resid = y_test - lasso.predict(X_test)

In [ ]:
sns.jointplot(ridge_resid, lasso_resid)

## Both seem normally distributed..and quite linear..

###  Lets go with Lasso

In [ ]:
lasso.fit(X_train, y_train)

In [ ]:
lasso_coefs = pd.DataFrame({'variable':TRAINSET.drop('SalePrice', axis=1).columns,
                            'coef':lasso.coef_,
                            'abs_coef':np.abs(lasso.coef_)})

lasso_coefs.sort_values('abs_coef', inplace=True, ascending=False)

lasso_coefs

In [ ]:
print 'Percent variables zeroed out:', np.sum((lasso.coef_ == 0))/float(len(lasso.coef_))

## REVISIT LinearRegression only using XXXXXXX's non-zero features.

In [ ]:
lasso_selected_feat = lasso_coefs[lasso_coefs.abs_coef != 0]['variable'].tolist()

In [ ]:
## Add in SalePrice and YrSold cols.
features_v2 = lasso_selected_feat + ['YrSold','SalePrice']

In [ ]:
## TRAININGDATA2 from TRAININGDATA
## (Reminder: TRAININGDATA IS ALREADY DUMMIED. EVERYTHING WAS DONE WITH DUMMIED OUT COLS)
TRAININGDATA2 = TRAININGDATA[features_v2].copy()

##SPLIT THE DATA INTO < 2010 AND == 2010
TESTSET2 = TRAININGDATA2[TRAININGDATA2.YrSold == 2010].copy()
TRAINSET2 = TRAININGDATA2[TRAININGDATA2.YrSold < 2010].copy()

In [ ]:
## YrSold served is Purpose, we can drop it.
TESTSET2.drop('YrSold', axis=1, inplace=True)
TRAINSET2.drop('YrSold', axis=1, inplace=True)

### TRAINSET2 & TESTSET2 !

In [ ]:
## TRAINING SET 2
X_train2 = TRAINSET2.drop('SalePrice', axis=1)
X_train2 = ss.fit_transform(X_train2) ##Standardize
y_train2 = TRAINSET2['SalePrice'].values

## TESTING SET 2
X_test2 = TESTSET2.drop('SalePrice', axis=1)
X_test2 = ss.fit_transform(X_test2) ##Standardize
y_test2 = TESTSET2['SalePrice'].values

In [ ]:
model2 = LinearRegression()
model2.fit(X_train2, y_train2)

In [ ]:
linreg_scores2 = cross_val_score(model2, X_test2, y_test2, cv=9) 
#print linreg_scores
print('LinearRegression with select features after Lasso: R2 score: {}'.format(np.mean(linreg_scores2)))

## Lets try it out

### Lets Predict with X_test2! on model2 !

In [ ]:
## Keep fingers crossed.
y_pred = model2.predict(X_test2)

In [ ]:
print('Mean squared error: {}'.format(mean_squared_error(y_test, y_pred)))
print('Model Score R2: {}'.format(model2.score(X_test2, y_test2)))
print('Cross Validated R2: {}'.format(np.mean(cross_val_score(model2, X_test2,y_test2, cv=9))))

In [ ]:
# Plot Residuals vs Predicted Residuals
plt.figure(figsize=(20,8))
sns.regplot(y_test, y_pred, fit_reg=True)

plt.title('Scatter Plot of Actual and Predicted Sales Prices', fontsize=20)
plt.xlabel('Predicted Sales Prices', fontsize=18)
plt.ylabel('Actual Sales Prices', fontsize=18)
plt.show()

In [ ]:
# Plot the Actual(y_test2) and Predicted (y_pred)
# BLUE = Actual (y_test2)
# RED =  Predicted (y_pred)

plt.figure(figsize=(20,8))
plt.scatter(range(len(y_test2)), y_test2,  color='cornflowerblue', label='Actual Prices (y_test2)') #Actual (y_test2)
plt.scatter(range(len(y_pred)), y_pred, color='red', alpha=0.5, label='Predicted Prices (y_pred)') #Predicted (y_pred)

plt.legend(loc='upper left', prop={'size': 16}, frameon=True)

plt.title('Visual difference between the Actual and Predicted', fontsize=20)
plt.xlabel('Houses Sold', fontsize=18)
plt.ylabel('Sale Prices', fontsize=18)
plt.show()

In [ ]:
# Plot the Residuals = y_test2 - y_pred
Residuals = y_test2 - y_pred

plt.figure(figsize=(20,5))
plt.scatter(range(len(Residuals)), Residuals,  color='cornflowerblue', label='Residuals = y_test2 - y_pred') #Residuals
#plt.scatter(range(len(y_pred)), y_pred, color='red', alpha=0.5, label='Predicted Prices (y_pred)') #Predicted (y_pred)

plt.legend(loc='upper left', prop={'size': 16}, frameon=True)

plt.title('Residuals', fontsize=20)
plt.xlabel('Houses Sold', fontsize=18)
plt.ylabel('Residuals', fontsize=18)
plt.axhline(y=0, color='darkorange')  ## the 0 line to see how far the residuals are.
plt.show()